In [4]:
import pandas as pd
data = pd.read_csv("bc_trip259172515_230215.csv")

In [5]:
filtered_data = data.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])

In [6]:
filtered_data.head(5)

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [7]:
filtered_data = pd.read_csv("bc_trip259172515_230215.csv", usecols=lambda column: column not in ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])

In [8]:
filtered_data.head(5)

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [13]:
from datetime import datetime, timedelta

def decode_timestamp(OPD_DATE, ACT_TIME):
    datetimen= datetime.strptime(OPD_DATE, "%d%b%Y:%H:%M:%S")
    timedeltat = timedelta(seconds=ACT_TIME)
    timestamp_col = datetimen + timedeltat
    return timestamp_col

filtered_data["TIMESTAMP"] = filtered_data.apply(lambda row: decode_timestamp(row["OPD_DATE"], row["ACT_TIME"]), axis=1)
filtered_data

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,2023-02-15 05:41:29
...,...,...,...,...,...,...,...,...
156,259172515,15FEB2023:00:00:00,4223,5834,21389,-122.677057,45.528040,2023-02-15 05:56:29
157,259172515,15FEB2023:00:00:00,4223,5838,21424,-122.677003,45.528037,2023-02-15 05:57:04
158,259172515,15FEB2023:00:00:00,4223,5858,21429,-122.676765,45.528043,2023-02-15 05:57:09
159,259172515,15FEB2023:00:00:00,4223,5889,21434,-122.676370,45.528047,2023-02-15 05:57:14


In [14]:
after_timestamp_data = filtered_data.drop(columns=['OPD_DATE', 'ACT_TIME'])
after_timestamp_data

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29
...,...,...,...,...,...,...
156,259172515,4223,5834,-122.677057,45.528040,2023-02-15 05:56:29
157,259172515,4223,5838,-122.677003,45.528037,2023-02-15 05:57:04
158,259172515,4223,5858,-122.676765,45.528043,2023-02-15 05:57:09
159,259172515,4223,5889,-122.676370,45.528047,2023-02-15 05:57:14


In [15]:
after_timestamp_data['dMETERS'] = after_timestamp_data['METERS'].diff()
after_timestamp_data['dTIMESTAMP'] = after_timestamp_data['TIMESTAMP'].diff().dt.total_seconds()
after_timestamp_data['SPEED'] = after_timestamp_data.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'], axis=1)
after_timestamp_data.iloc[0, after_timestamp_data.columns.get_loc('SPEED')] = after_timestamp_data.iloc[1]['SPEED']

after_timestamp_data.drop(['dMETERS', 'dTIMESTAMP'], axis=1, inplace=True)
after_timestamp_data

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,SPEED
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,1.600000
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,1.600000
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,1.800000
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,3.200000
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,7.800000
...,...,...,...,...,...,...,...
156,259172515,4223,5834,-122.677057,45.528040,2023-02-15 05:56:29,2.800000
157,259172515,4223,5838,-122.677003,45.528037,2023-02-15 05:57:04,0.114286
158,259172515,4223,5858,-122.676765,45.528043,2023-02-15 05:57:09,4.000000
159,259172515,4223,5889,-122.676370,45.528047,2023-02-15 05:57:14,6.200000


In [17]:
speed_stats = after_timestamp_data['SPEED'].describe()

print("Minimum speed:", speed_stats['min'], "m/s")
print("Maximum speed:", speed_stats['max'], "m/s")
print("Average speed:", speed_stats['mean'], "m/s")

Minimum speed: 0.0 m/s
Maximum speed: 17.4 m/s
Average speed: 7.192254226105158 m/s


In [1]:
import pandas as pd
day_data = pd.read_csv("bc_veh4223_230215.csv")

In [5]:
grouped_large = day_data.groupby('EVENT_NO_TRIP')
dfs_large = []
for name, group_large in grouped_large:

    group_large = group_large.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])
    group_large['TIMESTAMP'] = pd.to_datetime(group_large['OPD_DATE'], format='%d%b%Y:%H:%M:%S') + pd.to_timedelta(group_large['ACT_TIME'], unit='s')
    group_large['dTIMESTAMP'] = group_large['TIMESTAMP'].diff()
    group_large['dMETERS'] = group_large['METERS'].diff()
    group_large['SPEED'] = group_large['dMETERS'] / group_large['dTIMESTAMP'].dt.total_seconds()
    group_large = group_large.drop(columns=['dTIMESTAMP', 'dMETERS'])
    dfs_large.append(group_large)
df_large_transformed = pd.concat(dfs_large)


In [6]:
print(df_large_transformed.head())

      EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
3617      259169000  15FEB2023:00:00:00        4029    4223     25641   
2313      259172515  15FEB2023:00:00:00        4223      40     20469   
2314      259172515  15FEB2023:00:00:00        4223      48     20474   
2315      259172515  15FEB2023:00:00:00        4223      57     20479   
2316      259172515  15FEB2023:00:00:00        4223      73     20484   

      GPS_LONGITUDE  GPS_LATITUDE           TIMESTAMP  SPEED  
3617    -122.684340     45.508072 2023-02-15 07:07:21    NaN  
2313    -122.648137     45.493082 2023-02-15 05:41:09    NaN  
2314    -122.648240     45.493070 2023-02-15 05:41:14    1.6  
2315    -122.648352     45.493123 2023-02-15 05:41:19    1.8  
2316    -122.648385     45.493262 2023-02-15 05:41:24    3.2  


In [7]:
max_speed = df_large_transformed['SPEED'].max()

In [8]:
print("\nMaximum speed for vehicle #4223 on February 15, 2023:", max_speed)


Maximum speed for vehicle #4223 on February 15, 2023: 17.4


In [9]:
max_speed_row_large = df_large_transformed[df_large_transformed['SPEED'] == max_speed].iloc[0]
print("Location (Longitude, Latitude):", max_speed_row_large['GPS_LONGITUDE'], max_speed_row_large['GPS_LATITUDE'])
print("Time of occurrence:", max_speed_row_large['TIMESTAMP'])

Location (Longitude, Latitude): -122.660822 45.505452
Time of occurrence: 2023-02-15 05:44:49


In [10]:
median_speed_large = df_large_transformed['SPEED'].median()
print("\nMedian speed for vehicle #4223 on February 15, 2023:", median_speed_large)


Median speed for vehicle #4223 on February 15, 2023: 7.2


In [1]:
import pandas as pd

# Step 1: Read the complete dataset into a DataFrame
full_data = pd.read_csv("bc_230215.csv")

# Step 2: Remove unnecessary columns
columns_to_drop = ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP']
full_data.drop(columns=columns_to_drop, inplace=True)

# Step 3: Decode the timestamp
def decode_timestamp(row):
    opd_date = pd.to_datetime(row['OPD_DATE'], format='%d%b%Y:%H:%M:%S')
    act_time = pd.to_timedelta(row['ACT_TIME'], unit='s')
    return opd_date + act_time

full_data['TIMESTAMP'] = full_data.apply(decode_timestamp, axis=1)

# Step 4: Calculate time difference and distance traveled
full_data['TIME_DIFF'] = full_data.groupby('EVENT_NO_TRIP')['TIMESTAMP'].diff()
full_data['DIST_DIFF'] = full_data.groupby('EVENT_NO_TRIP')['METERS'].diff()

# Step 5: Calculate speed
full_data['SPEED'] = full_data['DIST_DIFF'] / full_data['TIME_DIFF'].dt.total_seconds()

# Step 6: Remove intermediate columns
full_data.drop(columns=['TIME_DIFF', 'DIST_DIFF'], inplace=True)

# Analyze the data to answer questions related to the full dataset
# For example:
# What was the maximum speed for any vehicle on February 15, 2023?
max_speed = full_data['SPEED'].max()
print("Maximum speed for any vehicle:", max_speed)

# Where and when did this maximum speed occur?
max_speed_row = full_data[full_data['SPEED'] == max_speed].iloc[0]
print("Location:", max_speed_row['GPS_LATITUDE'], max_speed_row['GPS_LONGITUDE'])
print("Timestamp:", max_speed_row['TIMESTAMP'])

# Which vehicle had the fastest mean speed for any single trip on this day?
fastest_vehicle_trip = full_data.groupby(['VEHICLE_ID', 'EVENT_NO_TRIP'])['SPEED'].mean().idxmax()
fastest_vehicle_id = fastest_vehicle_trip[0]
fastest_trip_id = fastest_vehicle_trip[1]
print("Fastest vehicle:", fastest_vehicle_id)
print("Fastest trip:", fastest_trip_id)

Maximum speed for any vehicle: 43.0
Location: 45.490948 -122.648452
Timestamp: 2023-02-16 00:05:18
Fastest vehicle: 3419
Fastest trip: 258614729
